In [10]:
# %%
import os
import pickle
import pathlib
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from collections import defaultdict
from sklearn.preprocessing import normalize
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.cluster import (
    KMeans, DBSCAN, AgglomerativeClustering
)
import hdbscan
import umap
import matplotlib.pyplot as plt

# %%
# Настройка путей
DATA_PATH = "/content/drive/MyDrive/testing_models"
EMB_DIR = pathlib.Path(DATA_PATH)
CLUSTER_DIR = pathlib.Path("/content/drive/MyDrive/testing_models/results/clusters")
CENTRAL_DIR = pathlib.Path("/content/drive/MyDrive/testing_models/results/reports")
VIS_DIR = pathlib.Path("/content/drive/MyDrive/testing_models/results/visualizations")

# Создание директорий
for dir in [CLUSTER_DIR, CENTRAL_DIR, VIS_DIR]:
    dir.mkdir(parents=True, exist_ok=True)

# %%
def load_embeddings(emb_dir):
    """Загрузка эмбеддингов из CSV-файлов"""
    embeddings = {}
    for emb_file in emb_dir.glob("*.csv"):
        if "labels" not in emb_file.stem and "report" not in emb_file.stem:
            name = emb_file.stem
            emb_df = pd.read_csv(emb_file, index_col="id")
            embeddings[name] = emb_df.values.astype(np.float32)
            print(f"Loaded {name}: {emb_df.shape}")
    return embeddings

embeddings = load_embeddings(EMB_DIR)
print(f"\nTotal embeddings loaded: {len(embeddings)}")

# %%
ALGORITHMS = {
    "kmeans": {
        "func": KMeans,
        "params": [
            {"n_clusters": n, "n_init": "auto", "random_state": 42}
            for n in range(5, 30, 2)  # Диапазон 3-30 с шагом 1
        ]
    },
    "dbscan": {
        "func": DBSCAN,
        "params": [
            {"eps": eps, "min_samples": ms, "metric": metric}
            for eps in np.linspace(0.3, 0.7, 3)
            for ms in range(5, 31, 2)  # min_samples от 3 до 30
            for metric in ["euclidean", "cosine"]
        ]
    },
    "hdbscan": {
        "func": hdbscan.HDBSCAN,
        "params": [
            {"min_cluster_size": mcs, "metric": metric, "cluster_selection_method": "eom"}
            for mcs in range(5, 31, 2)  # min_cluster_size от 3 до 30
            for metric in ["euclidean", "manhattan"]
        ]
    },
    "agglo": {
        "func": AgglomerativeClustering,
        "params": [
            {"n_clusters": n, "linkage": link}
            for n in range(5, 31, 2)  # Диапазон 3-30 с шагом 1
            for link in ["ward", "complete", "average"]
        ]
    }
}

# %%
def save_pkl(obj, path):
    """Сохранение объекта в pickle-файл"""
    with open(path, 'wb') as f:
        pickle.dump(obj, f)
    print(f"Saved {path.name}")

def load_pkl(path):
    """Загрузка объекта из pickle-файла"""
    with open(path, 'rb') as f:
        return pickle.load(f)

# %%
def cluster_and_report(name, X, algo_key, params):
    """Выполнение кластеризации и сохранение результатов"""
    try:
        # Создаем уникальный идентификатор эксперимента
        param_str = "_".join(f"{k}={v}" for k,v in params.items())
        experiment_id = f"{name}_{algo_key}_{param_str}"

        # Инициализация модели
        model = ALGORITHMS[algo_key]["func"](**params)
        labels = model.fit_predict(X)

        # Расчет метрик
        n_clusters = len(np.unique(labels[labels != -1]))
        metrics = {
            "n_clusters": n_clusters,
            "noise_ratio": np.mean(labels == -1),
            "silhouette": silhouette_score(X, labels) if n_clusters > 1 else np.nan,
            "davies_bouldin": davies_bouldin_score(X, labels) if n_clusters > 1 else np.nan
        }

        # Сохранение результатов
        results = {
            "experiment_id": experiment_id,
            "emb_name": name,
            "algorithm": algo_key,
            "params": params,
            "metrics": metrics,
            "labels": labels,
            "model": model
        }

        # Сохранение в pickle
        save_pkl(results, CLUSTER_DIR / f"{experiment_id}.pkl")

        return metrics

    except Exception as e:
        print(f"⚠️ Error in {algo_key}: {str(e)}")
        return None

# %%
def visualize_and_save(results, X):
    """Визуализация и сохранение графиков"""
    try:
        reducer = umap.UMAP(n_components=2, random_state=42,
                          n_neighbors=15, min_dist=0.1)
        emb2d = reducer.fit_transform(X)

        plt.figure(figsize=(12, 8))
        scatter = plt.scatter(emb2d[:,0], emb2d[:,1], s=15,
                            c=results['labels'], cmap='Spectral', alpha=0.8)
        plt.title(f"{results['experiment_id']}\nClusters: {results['metrics']['n_clusters']}")
        plt.colorbar(scatter)

        vis_path = VIS_DIR / f"{results['experiment_id']}_viz.png"
        plt.savefig(vis_path, dpi=120, bbox_inches='tight')
        plt.close()

        # Сохранение UMAP проекции
        umap_results = {
            "emb2d": emb2d,
            "labels": results['labels'],
            "experiment_id": results['experiment_id']
        }
        save_pkl(umap_results, VIS_DIR / f"{results['experiment_id']}_umap.pkl")

    except Exception as e:
        print(f"⚠️ Visualization failed: {str(e)}")

# %%
# Основной цикл обработки
all_results = []
for emb_name, X in embeddings.items():
    print(f"\n🔨 Processing embedding: {emb_name}")

    for algo_key in ALGORITHMS:
        print(f"\n🔄 Algorithm: {algo_key.upper()}")

        for params in tqdm(ALGORITHMS[algo_key]["params"], desc="Parameters"):
            result = cluster_and_report(emb_name, X, algo_key, params)

            if result:
                # Загрузка последних сохраненных результатов
                experiment_id = f"{emb_name}_{algo_key}_{'_'.join(f'{k}={v}' for k,v in params.items())}"
                saved_results = load_pkl(CLUSTER_DIR / f"{experiment_id}.pkl")

                # Визуализация и дополнительное сохранение
                visualize_and_save(saved_results, X)
                all_results.append(saved_results)

# %%
# Финальное сохранение всех результатов
final_report = {
    "all_experiments": all_results,
    "summary_stats": pd.DataFrame([r['metrics'] for r in all_results]).describe()
}

save_pkl(final_report, EMB_DIR.parent / "final_report.pkl")
print("✅ All operations completed!")

Loaded all-MiniLM-L6-v2: (375, 384)
Loaded all-mpnet-base-v2: (375, 768)
Loaded bge-base-en-v1.5: (375, 768)
Loaded distil-log: (375, 768)
Loaded bge-large-en-v1.5: (375, 1024)
Loaded e5-large-v2: (375, 1024)
Loaded tfidf: (375, 5000)
Loaded nomic-embed-text-v1.5: (375, 768)

Total embeddings loaded: 8

🔨 Processing embedding: all-MiniLM-L6-v2

🔄 Algorithm: KMEANS


Parameters:   0%|          | 0/13 [00:00<?, ?it/s]

Saved all-MiniLM-L6-v2_kmeans_n_clusters=5_n_init=auto_random_state=42.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Saved all-MiniLM-L6-v2_kmeans_n_clusters=5_n_init=auto_random_state=42_umap.pkl
Saved all-MiniLM-L6-v2_kmeans_n_clusters=7_n_init=auto_random_state=42.pkl


...

Saved nomic-embed-text-v1.5_agglo_n_clusters=29_linkage=ward_umap.pkl
Saved nomic-embed-text-v1.5_agglo_n_clusters=29_linkage=complete.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Saved nomic-embed-text-v1.5_agglo_n_clusters=29_linkage=complete_umap.pkl
Saved nomic-embed-text-v1.5_agglo_n_clusters=29_linkage=average.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Saved nomic-embed-text-v1.5_agglo_n_clusters=29_linkage=average_umap.pkl
Saved final_report.pkl
✅ All operations completed!


# Формирование отчета по всем результатам

In [8]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.6 MB/s eta 0:00:00


In [11]:
import pandas as pd
import pickle
from pathlib import Path
from tqdm import tqdm


# Настройка путей
RESULTS_DIR = Path("/content/drive/MyDrive/testing_models/results/clusters")
OUTPUT_PATH = Path("/content/drive/MyDrive/testing_models/clustering_metrics_report.xlsx")

def load_results(pkl_file):
    """Загрузка результатов из pickle-файла"""
    with open(pkl_file, 'rb') as f:
        data = pickle.load(f)
    return data

def parse_results(results):
    """Парсинг результатов эксперимента"""
    return {
        'embedding': results['emb_name'],
        'algorithm': results['algorithm'],
        'parameters': str(results['params']),
        'n_clusters': results['metrics']['n_clusters'],
        'noise_ratio': results['metrics']['noise_ratio'],
        'silhouette': results['metrics']['silhouette'],
        'davies_bouldin': results['metrics']['davies_bouldin'],
        'experiment_id': results['experiment_id']
    }

def generate_report(results_dir):
    """Генерация сводного отчета"""
    all_results = []

    # Получаем список всех pickle-файлов
    pkl_files = list(results_dir.glob("*.pkl"))

    for pkl_file in tqdm(pkl_files, desc="Processing files"):
        try:
            result_data = load_results(pkl_file)
            parsed = parse_results(result_data)
            all_results.append(parsed)
        except Exception as e:
            print(f"\n⚠️ Error processing {pkl_file.name}: {str(e)}")
            continue

    # Создаем DataFrame
    df = pd.DataFrame(all_results)

    # Оптимизация порядка столбцов
    column_order = [
        'experiment_id',
        'embedding',
        'algorithm',
        'n_clusters',
        'noise_ratio',
        'silhouette',
        'davies_bouldin',
        'parameters'
    ]

    return df[column_order]

def save_report(df, output_path):
    """Сохранение отчета с форматированием"""
    writer = pd.ExcelWriter(output_path, engine='xlsxwriter')

    # Сохраняем в Excel с автоподбором ширины колонок
    df.to_excel(writer, index=False, sheet_name='Metrics')

    # Настройка форматов
    workbook = writer.book
    worksheet = writer.sheets['Metrics']

    # Формат для числовых колонок
    num_format = workbook.add_format({'num_format': '0.000'})

    # Установка ширины колонок
    col_widths = {
        'experiment_id': 40,
        'embedding': 20,
        'algorithm': 15,
        'n_clusters': 12,
        'noise_ratio': 15,
        'silhouette': 15,
        'davies_bouldin': 15,
        'parameters': 50
    }

    for idx, col in enumerate(df.columns):
        worksheet.set_column(idx, idx, col_widths.get(col, 20))

    # Применение формата к числовым колонкам
    numeric_cols = ['noise_ratio', 'silhouette', 'davies_bouldin']
    for col in numeric_cols:
        col_idx = df.columns.get_loc(col)
        worksheet.set_column(col_idx, col_idx, None, num_format)

    # Добавление условного форматирования
    format_green = workbook.add_format({'bg_color': '#C6EFCE',
                                       'font_color': '#006100'})
    format_red = workbook.add_format({'bg_color': '#FFC7CE',
                                    'font_color': '#9C0006'})

    # Для silhouette score (чем выше, тем лучше)
    worksheet.conditional_format(
        'F2:F1048576',
        {'type': '3_color_scale',
         'min_type': 'num',
         'mid_type': 'num',
         'max_type': 'num',
         'min_value': -1,
         'mid_value': 0,
         'max_value': 1}
    )

    # Для davies_bouldin (чем ниже, тем лучше)
    worksheet.conditional_format(
        'G2:G1048576',
        {'type': '3_color_scale',
         'min_type': 'num',
         'mid_type': 'num',
         'max_type': 'num',
         'min_value': 0,
         'mid_value': 1,
         'max_value': 3,
         'reverse': True}
    )

    writer.close()

if __name__ == "__main__":
    # Генерация и сохранение отчета
    report_df = generate_report(RESULTS_DIR)

    # Дополнительная сортировка
    report_df = report_df.sort_values(
        by=['algorithm', 'silhouette'],
        ascending=[True, False]
    )

    # Сохранение в Excel
    save_report(report_df, OUTPUT_PATH)

    # Вывод сводки
    print("\n✅ Report generated successfully!")
    print(f"Total experiments processed: {len(report_df)}")
    print(f"Saved to: {OUTPUT_PATH}")
    print("\nTop 5 best results by Silhouette Score:")
    print(report_df.head(5).to_string(index=False))

Processing files: 100%|██████████| 1384/1384 [00:56<00:00, 24.69it/s]
/usr/local/lib/python3.11/dist-packages/xlsxwriter/worksheet.py:2754: UserWarning: Unknown parameter 'reverse' in conditional_format()
  warn(f"Unknown parameter '{param_key}' in conditional_format()")



✅ Report generated successfully!
Total experiments processed: 1384
Saved to: /content/drive/MyDrive/testing_models/clustering_metrics_report.xlsx

Top 5 best results by Silhouette Score:
                                 experiment_id  embedding algorithm  n_clusters  noise_ratio  silhouette  davies_bouldin                               parameters
 distil-log_agglo_n_clusters=5_linkage=average distil-log     agglo           5          0.0    0.854986        0.371118  {'n_clusters': 5, 'linkage': 'average'}
 distil-log_agglo_n_clusters=7_linkage=average distil-log     agglo           7          0.0    0.645982        0.360619  {'n_clusters': 7, 'linkage': 'average'}
distil-log_agglo_n_clusters=10_linkage=average distil-log     agglo          10          0.0    0.575113        0.416575 {'n_clusters': 10, 'linkage': 'average'}
 distil-log_agglo_n_clusters=9_linkage=average distil-log     agglo           9          0.0    0.572497        0.434356  {'n_clusters': 9, 'linkage': 'average'}
di

# Формирование файла csv (для каждого номера кластера записываем текст кластера)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
import hdbscan

# Ваши данные (замените на реальные)
embeddings = np.random.rand(100, 10)  # Пример эмбедингов
texts = [f"Text {i}" for i in range(100)]  # Пример текстов
df = pd.DataFrame({"errors": texts})  # DataFrame с текстами

ALGORITHMS = {
    "kmeans": {
        "func": KMeans,
        "params": [{"n_clusters": n, "n_init": "auto", "random_state": 42} for n in range(3, 31)]
    },
    "dbscan": {
        "func": DBSCAN,
        "params": [{"eps": eps, "min_samples": ms, "metric": metric}
                 for eps in np.linspace(0.3, 0.7, 3)
                 for ms in range(3, 31)
                 for metric in ["euclidean", "cosine"]]
    },
    "hdbscan": {
        "func": hdbscan.HDBSCAN,
        "params": [{"min_cluster_size": mcs, "metric": metric, "cluster_selection_method": "eom"}
                 for mcs in range(3, 31)
                 for metric in ["euclidean", "manhattan"]]
    },
    "agglo": {
        "func": AgglomerativeClustering,
        "params": [{"n_clusters": n, "linkage": link}
                 for n in range(3, 31)
                 for link in ["ward", "complete", "average"]]
    }
}

for algo_name, config in ALGORITHMS.items():
    model_func = config["func"]
    params_list = config["params"]

    for params in params_list:
        try:
            # Инициализация и обучение модели
            model = model_func(**params)
            labels = model.fit_predict(embeddings)

            # Создание DataFrame с кластерами
            cluster_data = []
            for cluster_id in np.unique(labels):
                # Получаем индексы объектов кластера
                cluster_indices = np.where(labels == cluster_id)[0]

                # Собираем тексты
                cluster_texts = [df.iloc[ix]["errors"] for ix in cluster_indices]

                # Формируем запись
                cluster_data.append({
                    "cluster_id": cluster_id,
                    "texts": " | ".join(cluster_texts)
                })

            # Создаем и сохраняем DataFrame
            result_df = pd.DataFrame(cluster_data)
            param_str = "_".join([f"{k}={v}" for k, v in params.items()])
            filename = f"{algo_name}_clusters_{param_str}.csv"
            result_df.to_csv(filename, index=False, encoding='utf-8')

            print(f"✓ Saved: {filename}")

        except Exception as e:
            print(f"⚠ Error with {algo_name} {params}: {str(e)}")